In [ ]:
# Import the basic libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

## Load the Data

In [ ]:
# Import funcitons for the velocity calculation; Correlate all the beams from one set of repeat ground tracks, rgt = 0848
from IS2_velocity.correlation_processing import calculate_velocities
from IS2_velocity.readers import load_data_by_rgt
from scipy.signal import correlate
from astropy.time import Time

### Data directory
data_dir = '../data/'

### Select rgt for now
rgt = '0848'

### Control the correlation step:
segment_length = 2000 # meters, how wide is the window we are correlating in each step
search_width = 1000 # meters, how far in front of and behind the window to check for correlation
along_track_step = 100 # meters; how much to jump between each consecutivevelocity determination
max_percent_nans = 10 # Maximum % of segment length that can be nans and still do the correlation step

### Which product
product = 'ATL06'
if product == 'ATL06':
    dx = 20

### Select filter type and required arguments; Currently only this running mean is supported
filter_type = 'running_average'
running_avg_window = 100 # meters

### Load data
x_atc, lats, lons, h_li_raw, h_li_raw_NoNans, h_li, h_li_diff, times, min_seg_ids, \
segment_ids, cycles_this_rgt, x_ps, y_ps = \
    load_data_by_rgt(rgt = rgt, path_to_data = data_dir, product = 'ATL06', \
                     filter_type = filter_type, running_avg_window = running_avg_window, \
                     format = 'hdf5')

## Correlation

In [ ]:
cycle1='03'
cycle2='04'
beam = 'gt1l'
start_ind = 3500

x_in1 = x_atc[cycle1][beam]
dh1 = h_li_diff[cycle1][beam]
dh2 = h_li_diff[cycle2][beam]

xi = start_ind
x_start = x_in1[start_ind]

# find indices for the stencil
idx1 = np.argmin(abs(x_in1-x_start))
idx2 = idx1 + int(np.round(segment_length/dx))
# cut out a wider chunk of data at time t2 (second cycle)
idx3 = idx1 - int(np.round(search_width/dx)) # offset on earlier end by # indices in search_width
idx4 = idx2 + int(np.round(search_width/dx)) # offset on later end by # indices in search_width

# get the two arrays that will be correlated to one another
dh_stencil = dh1[idx1:idx2]
dh_wide = dh2[idx3:idx4]

# correlate old with newer data
corr = correlate(dh_stencil, dh_wide, mode = 'valid', method = 'direct')
norm_val = np.sqrt(np.sum(dh_stencil**2)*np.sum(dh_wide**2)) # normalize so values range between 0 and 1
corr = corr / norm_val
lagvec = np.arange(- int(np.round(search_width/dx)), int(search_width/dx) +1,1)# for mode = 'valid'
shift_vec = lagvec * dx

t1_string = times[cycle1]['gt1l'][0].astype(str)  # figure out later if just picking hte first one it ok
t1 = Time(t1_string)
t2_string = times[cycle2]['gt1l'][0].astype(str)  # figure out later if just picking hte first one it ok
t2 = Time(t2_string)
dt = (t2 - t1).jd  # difference in julian days

## Plot with Widget

In [ ]:
import ipywidgets

x0 = x_atc[cycle1][beam][start_ind]
search_x = int(np.round(search_width/dx))
               
plt.figure(figsize=(8,5))
plt.subplot(221)
plt.tick_params(labelbottom=False)
l1, = plt.plot((x_atc[cycle1][beam][idx1:idx2]-x0-search_width)/1000.,h_li[cycle1][beam][idx1:idx2])
plt.plot((x_atc[cycle2][beam][idx3:idx4]-x0)/1000.,h_li[cycle2][beam][idx3:idx4])
plt.ylabel('Elevation (m)')

plt.subplot(223)
l2, = plt.plot((x_atc[cycle1][beam][idx1:idx2]-x0-search_width)/1000.,h_li_diff[cycle1][beam][idx1:idx2])
plt.plot((x_atc[cycle2][beam][idx3:idx4]-x0)/1000.,h_li_diff[cycle2][beam][idx3:idx4])
plt.ylabel('Slope (m/m)')
plt.xlabel('Along-Track Distance (km)')

plt.subplot(224)
plt.plot(shift_vec/(dt/365.),corr,'k')
p1, = plt.plot(shift_vec[0]/(dt/365.),corr[0],'k.',mfc='w',mew=2,ms=10)
plt.ylabel('Correlation')
plt.xlabel('Velocity (m/yr)')

plt.tight_layout()

# An update function for the interactive figure
def update(n_step = (0,search_x*2)):
    l1.set_xdata((x_atc[cycle1][beam][idx1:idx2]-x0+search_width-n_step*dx)/1000.)
    l2.set_xdata((x_atc[cycle1][beam][idx1:idx2]-x0+search_width-n_step*dx)/1000.)
    p1.set_data(np.transpose([shift_vec[int(n_step)]/(dt/365.),corr[int(n_step)]]))

# Update with the sliders
ipywidgets.interactive(update)